# Recommendation Systems

Recommendation systems have revolutionized the way we discover and explore new things. These intelligent systems utilize sophisticated algorithms and data analysis to understand individual preferences and provide personalized recommendations. 
<br><br>
By analyzing user data, such as **browsing history**, **purchase patterns**, and **social interactions**, recommendation systems can effectively predict and suggest items that align with users' interests. Whether it's suggesting a new movie to watch, a book to read, or a product to buy, these systems streamline decision-making and enhance the overall user experience. 
<br>
With their ability to uncover hidden gems and introduce users to exciting possibilities, recommendation systems have become invaluable tools in navigating the overwhelming abundance of choices in today's digital landscape.

### Recommendation Systems and Vector Databases

In recommendation systems, understanding the similarity between users and items is crucial for generating accurate and personalized recommendations. By leveraging vector databases, these systems can store and organize user and item vectors, which capture the essential characteristics and preferences associated with each user and item. 
<br><br>
The vector database employs <a href="https://www.pinecone.io/learn/vector-database/#:~:text=a%20vector%20database.-,Algorithms,-Several%20algorithms%20can">advanced indexing techniques</a>, to enable fast retrieval of **similar users or items** based on their **vector representations**. This enables recommendation systems to efficiently process *large-scale datasets* and identify meaningful connections, leading to more precise and relevant recommendations. 
<br><br>
By harnessing the power of vector databases, such as **Pinecone**, recommendation systems can optimize their performance, enhance user satisfaction, and deliver tailored experiences that align with individual preferences.
<br><br><br>
Let's take a look at how we can implement one of those use cases!

We start by installing all necessary libraries.

In [ ]:
!pip install -qU \
    "pinecone-client[grpc]"==2.2.1 \
    pinecone-datasets \
    transformers \
    tensorflow

---

🚨 _Note: the above `pip install` is formatted for Jupyter notebooks. If running elsewhere you may need to drop the `!`._

---

## Data Preparation

<img alt="Onboarding recommender diagram" src="https://github.com/pinecone-io/examples/blob/onboarding-recommender/recommendation/onboarding-recommender/assets/onboarding_recommender_data_flow.jpg"  width="70%">

#### Downloading the Dataset

We will download a pre-embedding dataset from `pinecone-datasets`. Allowing us to skip the embedding and any other preprocessing steps. 
<br><br>
When working with your own dataset you will need to perform this embedding step but we have prebuilt the embeddings so we can jump right to the action.

In [ ]:
DATASET_NAME = "movielens-user-ratings"

In [ ]:
from pinecone_datasets import load_dataset

dataset = load_dataset(DATASET_NAME)
dataset.head()

In [ ]:
len(dataset)

#### Reformatting the Dataset

A `pinecone-dataset` always contains `id`, `values`, `sparse_values`, `metadata`, and `blob`. All we need are the IDs, vector embeddings (stored in `values`), and some metadata (which is actually stored in `blob`). Let's reformat the dataset ready for adding to Pinecone. We also drop `sparse_values` as they are not needed for this example.


In [ ]:
dataset.documents.drop(['sparse_values', 'metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)

dataset.head()

Here is an example of the metadata value.

In [ ]:
from pprint import pp

pp(dataset.documents['metadata'][0])

Now we move on to initializing our Pinecone vector database.

## Creating an Index

In [ ]:
import os
import pinecone
import time

We set `PINECONE_API_KEY` and `PINECONE_ENV` variables that we are going to use during initialization step. You can find these values in [Pinecone Console](https://app.pinecone.io/) in the API Keys section.

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

Now we can use these variables for initialization.

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In order to create a new index, we need to specify the index name, similarity metric, as well as the dimension of the vectors stored in that index. 
<br>
We will assign these values here. 
<br>
Note that the dimension parameter has to match the embedding dimensions provided in the dataset (or the model that outputs those embeddings).

In [ ]:
# embedding dimensions
len(dataset.documents['values'][0])

In [ ]:
INDEX_NAME = 'onboarding-recommender'
SIM_METRIC = 'cosine'
DIMENSION = 32

First, we need to check if the index already exists. In this example, we will delete it and create a new one.

In [ ]:
if INDEX_NAME in pinecone.list_indexes():
    pinecone.delete_index(INDEX_NAME)

pinecone.create_index(
    name=INDEX_NAME,
    metric=SIM_METRIC,
    dimension=DIMENSION,
)
# wait a moment for the index to be fully initialized
time.sleep(1)

We are going to initialize an index variable so that we can use it later on to describe the index and perform vector upsert.

In [ ]:
grpc_index = pinecone.GRPCIndex(INDEX_NAME)

In [ ]:
grpc_index.describe_index_stats()

In [ ]:
grpc_index.upsert_from_dataframe(dataset.documents)

In [ ]:
grpc_index.describe_index_stats()

## Querying the Index

Now, when the index is populated, we can perform queries on it to find the most relevant recommendations.
<br>
To do that, we need to instantiate our embedding models so that we can create vectors from our input user or input item objects.

### Getting the Model

We will download the models from the HuggingFace Hub. We will use one model to embed the *example user* and another model to embed the *example item*. <br>
This will allow us to retrieve the most relevant items for a specific user or find the most similar items to a specific item.

In [ ]:
from huggingface_hub import from_pretrained_keras

user_model = from_pretrained_keras("pinecone/movie-recommender-user-model")
movie_model = from_pretrained_keras("pinecone/movie-recommender-movie-model")

Before we proceed, we can create a `movie_details` dataset that we can use later on to print out the results.

In [ ]:
import pandas as pd

movies_details = pd.DataFrame(dataset.documents['metadata'].values.tolist())
movies_details.head()

#### Item Similarity

First, we can check how our vector database behaves when returning the most similar movies upon querying it using the movie vector created using the `movie_model` loaded above.

In [ ]:
movie_id = 1263  # you can try experimenting with different movie ids to obtain different results, for example 3571
movie_vector = movie_model(movie_id).numpy().tolist()

In [ ]:
movies_details[movies_details['movie_id'] == movie_id]['title'].tolist()[0]

In [ ]:
movie_query_results = grpc_index.query(queries=[movie_vector], top_k=10, include_metadata=True)

In [ ]:
for res in movie_query_results.results:
    df = pd.DataFrame(
        {
            'movies': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendations: ")
    display(df)

We can observe that it is doing an excellent job in finding similar movies, and it is accomplishing this task very quickly.

#### User Recommendations

Now, let's observe how our vector database behaves when we query it using the user vector.
<br>
We expect to receive movies that closely resemble the ones that the user rated highly.

In [ ]:
user_id = 3
user_vector = user_model(user_id).numpy().tolist()

Here, we are defining a function that allows us to easily display the movies that the user rated in the past.

In [ ]:
def top_movies_user_rated(user):
    # get list of movies that the user has rated
    user_movies = movies_details[movies_details["user_id"] == user]
    # order by their top rated movies
    top_rated = user_movies.sort_values(by=['rating'], ascending=False)
    # return the top 14 movies
    return pd.DataFrame(
        {
            'movies': top_rated['title'].tolist()[:14],
            'ratings': top_rated['rating'].tolist()[:14]
        }
    )

In [ ]:
display(top_movies_user_rated(user_id))

And now we can pass our `user_vector` to the query to get the recommendations.

In [ ]:
query_results = grpc_index.query(queries=[user_vector], top_k=10, include_metadata=True)

In [ ]:
for res in query_results.results:
    df = pd.DataFrame(
        {
            'movies': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendations: ")
    display(df)

Again, we can observe that these recommendations strongly resemble the movies that the user rated highly, and there are no movies similar to the ones that the user rated with a low value.

## Summary

The notebook demonstrated the step-by-step process of creating and populating an index in the vector database. It covered aspects such as specifying the index name, similarity metric, and vector dimensions. The example also included instructions on checking if an index exists, deleting and creating new indexes when necessary.

Furthermore, the notebook illustrated the usage of embedding models to generate vector representations of both users and items. The results showed that the recommendations closely resembled the movies that the user rated highly, and dissimilar movies were not included.

Overall, this example showcased the power and efficiency of vector databases in recommendation systems. It is important to note that the benefits of vector databases extend beyond movies and can be applied to various types of items, making them a valuable tool in building effective recommendation systems.